# Dependencies

## Install Dependencies

In [1]:
pip install sshtunnel pymysql pandas


[notice] A new release of pip available: 22.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Import Dependencies

In [2]:
from sshtunnel import SSHTunnelForwarder
import pymysql
import pandas as pd

In [3]:
# Konfigurasi koneksi SSH dan MySQL
SSH_HOST = 'kehadiran-bkd.pemkomedan.go.id'
SSH_PORT = 8570
SSH_USER = 'root'
SSH_PASSWORD = '53RV3R@4BS3NS1(123)'  # atau gunakan private key

DB_HOST = '127.0.0.1'  # karena kita tunneling ke lokal
DB_PORT = 3306
DB_USER = 'root'
DB_PASSWORD = 'DB_4BS3NS1BKD@2022'
DB_NAME = 'bkd_presensi'

# Get Data

## Insiasi Data

In [628]:
# Inisiasi Bulan dan Tahun Cetak
bulan_cetak = 10
tahun_cetak = 2025

# Insiasi instansi id
instansi_id = 3062  # Ganti dengan instansi_id yang sesuai

# satpol = 1206, bkpsdm = 3062 , bapenda = 3182

# Tanggal Awal dan Akhir Bulan
from calendar import monthrange
tanggal_awal = f"{tahun_cetak}{bulan_cetak:02d}01"
tanggal_akhir = f"{tahun_cetak}{bulan_cetak:02d}{monthrange(tahun_cetak, bulan_cetak)[1]}"

# Jika tanggal_awal dan tanggal_akhir masih string
tanggal_awal = pd.to_datetime(tanggal_awal)
tanggal_akhir = pd.to_datetime(tanggal_akhir)

In [629]:
tanggal_awal, tanggal_akhir

(Timestamp('2025-10-01 00:00:00'), Timestamp('2025-10-31 00:00:00'))

## Query Data

In [630]:
# Membuat tunnel SSH
with SSHTunnelForwarder(
    (SSH_HOST, SSH_PORT),
    ssh_username=SSH_USER,
    ssh_password=SSH_PASSWORD,  # atau gunakan parameter `ssh_pkey='/path/to/private/key.pem'`
    remote_bind_address=(DB_HOST, DB_PORT),
    local_bind_address=('127.0.0.1', 10022)  # port lokal bebas
) as tunnel:
    
    connection = pymysql.connect(
        host='127.0.0.1',
        port=10022,
        user=DB_USER,
        password=DB_PASSWORD,
        db=DB_NAME
    )

    # Query untuk mengambil data presensi karyawan
    # dengan filter instansi_id dan deleted_at sesuai bulan dan tahun cetak

    query_karyawan="""
    SELECT 
        * 
    FROM 
        bkd_presensi.presensi_karyawan 
    WHERE 1
        AND instansi_id = %s 
        AND (deleted_at IS NULL OR month(deleted_at) = %s AND year(deleted_at) = %s)
    """

    params = [instansi_id, bulan_cetak, tahun_cetak]

    df_pegawai = pd.read_sql(query_karyawan, connection, params=params)

    query_presensi = """
    SELECT 
        *
    FROM
        bkd_presensi.presensi_kehadiran
    WHERE 1
        AND instansi_id = %s
        AND date(tanggal_masuk) between %s and %s
    """

    params = [
        instansi_id,
        tanggal_awal.strftime('%Y-%m-%d'),
        tanggal_akhir.strftime('%Y-%m-%d')
    ]

    df_presensi = pd.read_sql(query_presensi, connection, params=params)

    query_rencana = """
    SELECT * FROM bkd_presensi.presensi_rencana_shift
    WHERE 1
        AND instansi_id = %s
        AND date(tanggal_masuk) between %s and %s
    """
    params = [
        instansi_id,
        tanggal_awal.strftime('%Y-%m-%d'),
        tanggal_akhir.strftime('%Y-%m-%d')
    ]

    df_rencana = pd.read_sql(query_rencana, connection, params=params)

    query_shift = """
    SELECT 
        *
    FROM
        presensi_shift
    """

    df_shift = pd.read_sql(query_shift, connection)

    query_absens = """
    SELECT 
        presensi_absen.*
    FROM
        presensi_absen
    LEFT JOIN 
        presensi_karyawan ON presensi_absen.karyawan_id = presensi_karyawan.id
    WHERE 
        instansi_id = %s
    AND (
        (tanggal_mulai >= %s AND tanggal_selesai <= %s)
        OR (tanggal_mulai BETWEEN %s AND %s)
        OR (tanggal_selesai BETWEEN %s AND %s)
        OR (%s BETWEEN tanggal_mulai AND tanggal_selesai)
        OR (%s BETWEEN tanggal_mulai AND tanggal_selesai)
        )

    """

    params = [
        instansi_id,
        tanggal_awal.strftime('%Y-%m-%d'),
        tanggal_akhir.strftime('%Y-%m-%d'),
        tanggal_awal.strftime('%Y-%m-%d'),
        tanggal_akhir.strftime('%Y-%m-%d'),
        tanggal_awal.strftime('%Y-%m-%d'),
        tanggal_akhir.strftime('%Y-%m-%d'),
        tanggal_awal.strftime('%Y-%m-%d'),
        tanggal_akhir.strftime('%Y-%m-%d'),
    ]

    df_absen = pd.read_sql(query_absens, connection, params=params)

    connection.close()

### Data Pegawai

In [631]:
df_pegawai.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76 entries, 0 to 75
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id                   76 non-null     int64         
 1   nip                  76 non-null     object        
 2   name                 76 non-null     object        
 3   group_id             76 non-null     int64         
 4   imei                 76 non-null     object        
 5   instansi_id          76 non-null     int64         
 6   created_at           76 non-null     datetime64[ns]
 7   updated_at           76 non-null     datetime64[ns]
 8   deleted_at           0 non-null      object        
 9   tempat_lahir         76 non-null     object        
 10  tanggal_lahir        76 non-null     object        
 11  jenis_kelamin        76 non-null     object        
 12  pendidikan_terakhir  76 non-null     object        
 13  alamat               76 non-null     

In [632]:
df_pegawai.head()  # Menampilkan beberapa baris pertama dari DataFrame

,id,nip,name,group_id,imei,instansi_id,created_at,updated_at,deleted_at,tempat_lahir,...,golongan,kordinat,jabatan,eselon_id,pangkat_id,status_face,comment_face,verified_id,verified_date,presensi_face
0,2470,198412222003121001,"SUBHAN FAJRI HARAHAP, S.STP, M.A.P",1,69CD50B1EBD8C8F7FE0E6A752C782D0D,3062,2020-06-15 19:53:39,2025-11-04 08:47:00,None,MEDAN,...,Pembina Tk. I (IV/b),"{""lat"":""3.5578531"",""long"":""98.6464847"",""rad"":""...",Kepala Badan Kepegawaian dan Pengembangan Sumb...,4.0,14.0,0,None,NaN,NaT,1
1,14080,196803191996022001,Ir NETTI EFRIDAWATI PURBA,1,BA05C3850F99E6238CF2169A7B82355A,3062,2020-06-09 10:26:09,2025-11-04 08:47:00,None,KAB. TAPANULI SELATAN,...,Pembina (IV/a),None,Analis Sumber Daya Manusia Aparatur,0.0,13.0,2,None,829.0,2025-10-14 07:36:57,1
2,14108,196804041993032006,"NUR ASIAH RANGKUTI, SH",1,AF2D29CAB6F327423A99A794A2222B44,3062,2020-06-09 10:26:09,2025-11-04 08:47:00,None,KEDAI DURIAN,...,Pembina (IV/a),None,Analis Sumber Daya Manusia Aparatur,0.0,13.0,2,None,829.0,2025-10-14 07:37:07,1
3,14472,198611062011012007,"RIDHA NOVIANA HARAHAP, S.Kom, M.I.Kom",1,$2y$10$i99iGpxQGfyM4akpqDrEJ.klaWlhs5.2IIay0jn...,3062,2020-06-09 10:26:09,2025-11-04 08:47:00,None,SEI ROKAN,...,Penata (III/c),None,Kepala Sub Bagian Umum Badan Kepegawaian dan P...,7.0,11.0,2,None,829.0,2025-10-20 08:51:16,1
4,15486,197304212009022001,"ASLAMIAH, S.E.",1,$2y$10$In002uDyZ0bF.LHHscQSgu6JDdggxdlQx2fmZQf...,3062,2020-06-09 10:26:09,2025-11-04 08:47:00,None,PANYABUNGAN,...,Penata Tk. I (III/d),None,Analis Sumber Daya Manusia Aparatur,0.0,12.0,2,Tidak Ada Foto Hadap kanan dan hadap kiri,829.0,2025-10-14 11:23:07,1


In [633]:
#tampilkan jumlah data pegawai
print(f"Jumlah data pegawai: {len(df_pegawai)}")

Jumlah data pegawai: 76


### Data Presensi

In [634]:
# df_presensi['tanggal_masuk'] = pd.to_datetime(df_presensi['tanggal_masuk']).dt.date

In [635]:
df_presensi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3134 entries, 0 to 3133
Data columns (total 36 columns):
 #   Column             Non-Null Count  Dtype          
---  ------             --------------  -----          
 0   id                 3134 non-null   object         
 1   karyawan_id        3134 non-null   int64          
 2   instansi_id        3134 non-null   int64          
 3   jenis              3134 non-null   object         
 4   lat                3134 non-null   object         
 5   long               3134 non-null   object         
 6   status_lokasi      3134 non-null   int64          
 7   status             0 non-null      object         
 8   catatan            3134 non-null   object         
 9   approver_id        3134 non-null   int64          
 10  approver_at        3134 non-null   datetime64[ns] 
 11  approver_status    3134 non-null   object         
 12  tanggal_kirim      3134 non-null   datetime64[ns] 
 13  message            3134 non-null   object       

In [636]:
df_presensi.head()  # Menampilkan beberapa baris pertama dari DataFrame

,id,karyawan_id,instansi_id,jenis,lat,long,status_lokasi,status,catatan,approver_id,...,is_admin_device,tanggal_masuk,jam_masuk,jam_pulang,created_at,updated_at,absen_lokasi,lokasi_sekarang,appversion,verif_face
0,0006550d-fc60-4a7c-ba28-acc1e30f781c,26557,3062,P,3.5905045,98.6750764,1,None,,829,...,0,2025-10-28,0 days 07:30:00,0 days 16:00:00,2025-10-28 16:00:53,2025-10-29 07:18:26,None,None,2,1.0
1,000cce79-4e47-4421-ba31-e9529a698116,32760,3062,M,3.5746599,98.6723299,0,None,spt adc walikota,829,...,0,2025-10-08,0 days 08:30:00,0 days 17:00:00,2025-10-08 08:02:34,2025-10-08 08:03:21,None,None,1,NaN
2,000e018b-8159-4b22-a1ab-9ed59ee8c3f4,31139,3062,M,3.6157107,98.6589712,0,None,panitia pkp,829,...,0,2025-10-10,0 days 07:30:00,0 days 16:30:00,2025-10-10 07:01:46,2025-10-10 07:22:44,None,None,1,NaN
3,001acc96-eadc-4a8e-b95a-6d27c3e7c32e,20610,3062,P,3.6156982,98.6590362,0,None,spt pkp,829,...,0,2025-10-10,0 days 08:30:00,0 days 17:30:00,2025-10-10 17:53:36,2025-10-14 07:31:07,None,None,1,NaN
4,001baa2b-f7f4-4c31-8bbd-772596f01391,20057,3062,M,3.590672,98.6746312,1,None,,829,...,0,2025-10-20,0 days 08:00:00,0 days 16:30:00,2025-10-20 07:31:51,2025-10-20 08:16:41,None,None,2,NaN


### Rencana Shift

In [637]:
df_rencana.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1744 entries, 0 to 1743
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   karyawan_id    1744 non-null   int64         
 1   instansi_id    1744 non-null   int64         
 2   tanggal_masuk  1744 non-null   object        
 3   shift_id       1744 non-null   int64         
 4   created_at     1744 non-null   datetime64[ns]
 5   editor_id      1744 non-null   int64         
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 81.9+ KB


In [638]:
df_rencana.head()  # Menampilkan beberapa baris pertama dari DataFrame rencana shift

,karyawan_id,instansi_id,tanggal_masuk,shift_id,created_at,editor_id
0,2470,3062,2025-10-01,1,2025-09-29 15:12:22,829
1,2470,3062,2025-10-02,1,2025-09-29 15:12:22,829
2,2470,3062,2025-10-03,2,2025-09-29 15:08:31,829
3,2470,3062,2025-10-06,1,2025-09-29 15:12:22,829
4,2470,3062,2025-10-07,1,2025-09-29 15:12:22,829


### Ref Shift

In [639]:
df_shift.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76 entries, 0 to 75
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype          
---  ------            --------------  -----          
 0   id                76 non-null     int64          
 1   group_id          76 non-null     int64          
 2   name              76 non-null     object         
 3   masuk_pre_time    76 non-null     timedelta64[ns]
 4   masuk_post_time   76 non-null     timedelta64[ns]
 5   masuk_max_time    76 non-null     timedelta64[ns]
 6   pulang_pre_time   76 non-null     timedelta64[ns]
 7   pulang_post_time  76 non-null     timedelta64[ns]
 8   jam_masuk         76 non-null     timedelta64[ns]
 9   jam_pulang        76 non-null     timedelta64[ns]
dtypes: int64(2), object(1), timedelta64[ns](7)
memory usage: 6.1+ KB


In [640]:
df_shift.head()  # Menampilkan beberapa baris pertama dari DataFrame shift

,id,group_id,name,masuk_pre_time,masuk_post_time,masuk_max_time,pulang_pre_time,pulang_post_time,jam_masuk,jam_pulang
0,1,1,MDN01 - PEMKO MEDAN SENIN-KAMIS,0 days 07:30:00,0 days 08:15:59,0 days 12:00:59,0 days 16:30:00,0 days 17:00:59,0 days 08:00:00,0 days 16:30:00
1,2,1,MDN02 - PEMKO MEDAN JUMAT,0 days 07:30:00,0 days 08:15:59,0 days 12:00:59,0 days 17:00:00,0 days 17:30:59,0 days 08:00:00,0 days 17:00:00
2,5,1,HUB01 - DISHUB Senin-Kamis,0 days 07:55:00,0 days 08:15:59,0 days 11:00:59,0 days 15:45:00,0 days 16:30:59,0 days 08:00:00,0 days 15:45:00
3,6,1,HUB02 - DISHUB Jumat,0 days 07:55:00,0 days 08:15:59,0 days 12:00:59,0 days 16:15:00,0 days 17:00:59,0 days 08:00:00,0 days 16:15:00
4,7,1,HUB03 - DISHUB Sabtu,0 days 07:55:00,0 days 08:15:59,0 days 10:00:59,0 days 12:00:00,0 days 12:45:59,0 days 08:00:00,0 days 12:00:00


### Data Absen / Izin

In [641]:
df_absen.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 0 to 65
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id               66 non-null     int64         
 1   karyawan_id      66 non-null     int64         
 2   tanggal_mulai    66 non-null     object        
 3   tanggal_selesai  66 non-null     object        
 4   type             66 non-null     object        
 5   catatan          66 non-null     object        
 6   editor_id        66 non-null     int64         
 7   deleted_at       1 non-null      datetime64[ns]
 8   created_at       66 non-null     datetime64[ns]
 9   updated_at       66 non-null     datetime64[ns]
dtypes: datetime64[ns](3), int64(3), object(4)
memory usage: 5.3+ KB


In [642]:
df_absen.head()

,id,karyawan_id,tanggal_mulai,tanggal_selesai,type,catatan,editor_id,deleted_at,created_at,updated_at
0,52600,14108,2025-10-14,2025-10-14,S,,829,NaT,2025-10-17 14:48:59,2025-10-17 14:48:59
1,53069,14108,2025-10-22,2025-10-28,C,Cuti Alasan Penting,829,NaT,2025-10-30 07:21:21,2025-10-30 07:21:21
2,53430,14108,2025-10-31,2025-10-31,S,,829,NaT,2025-11-03 08:56:39,2025-11-03 08:56:39
3,53802,16443,2025-10-27,2025-11-09,C,Cuti Alasan Penting,829,NaT,2025-11-03 19:23:10,2025-11-03 19:23:10
4,53474,16640,2025-10-30,2025-10-30,S,,829,NaT,2025-11-03 09:43:03,2025-11-03 09:43:03


#### Convert Datetime

In [643]:
df_absen['tanggal_mulai'] = pd.to_datetime(df_absen['tanggal_mulai'])
df_absen['tanggal_selesai'] = pd.to_datetime(df_absen['tanggal_selesai'])

In [644]:
df_absen.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 0 to 65
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id               66 non-null     int64         
 1   karyawan_id      66 non-null     int64         
 2   tanggal_mulai    66 non-null     datetime64[ns]
 3   tanggal_selesai  66 non-null     datetime64[ns]
 4   type             66 non-null     object        
 5   catatan          66 non-null     object        
 6   editor_id        66 non-null     int64         
 7   deleted_at       1 non-null      datetime64[ns]
 8   created_at       66 non-null     datetime64[ns]
 9   updated_at       66 non-null     datetime64[ns]
dtypes: datetime64[ns](5), int64(3), object(2)
memory usage: 5.3+ KB


# Olah data kehadiran

### Pre-Processing

#### Merge df_rencana + df_shift

In [645]:
df_rencana_shift = df_rencana.merge(df_shift, left_on='shift_id', right_on='id')

df_rencana_shift.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1744 entries, 0 to 1743
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype          
---  ------            --------------  -----          
 0   karyawan_id       1744 non-null   int64          
 1   instansi_id       1744 non-null   int64          
 2   tanggal_masuk     1744 non-null   object         
 3   shift_id          1744 non-null   int64          
 4   created_at        1744 non-null   datetime64[ns] 
 5   editor_id         1744 non-null   int64          
 6   id                1744 non-null   int64          
 7   group_id          1744 non-null   int64          
 8   name              1744 non-null   object         
 9   masuk_pre_time    1744 non-null   timedelta64[ns]
 10  masuk_post_time   1744 non-null   timedelta64[ns]
 11  masuk_max_time    1744 non-null   timedelta64[ns]
 12  pulang_pre_time   1744 non-null   timedelta64[ns]
 13  pulang_post_time  1744 non-null   timedelta64[ns]
 14  jam_masu

In [646]:
# 1. Konversi kolom ke tipe yang benar
df_rencana_shift['tanggal_masuk'] = pd.to_datetime(df_rencana_shift['tanggal_masuk'])

# 2. Gabungkan menjadi kolom datetime
df_rencana_shift['masuk_pre_time'] = df_rencana_shift['tanggal_masuk'] + df_rencana_shift['masuk_pre_time']
df_rencana_shift['masuk_post_time'] = df_rencana_shift['tanggal_masuk'] + df_rencana_shift['masuk_post_time']
df_rencana_shift['masuk_max_time'] = df_rencana_shift['tanggal_masuk'] + df_rencana_shift['masuk_max_time']
df_rencana_shift['pulang_pre_time'] = df_rencana_shift['tanggal_masuk'] + df_rencana_shift['pulang_pre_time']
df_rencana_shift['pulang_post_time'] = df_rencana_shift['tanggal_masuk'] + df_rencana_shift['pulang_post_time']
df_rencana_shift['jam_masuk'] = df_rencana_shift['tanggal_masuk'] + df_rencana_shift['jam_masuk']
df_rencana_shift['jam_pulang'] = df_rencana_shift['tanggal_masuk'] + df_rencana_shift['jam_pulang']

df_rencana_shift['mask'] = df_rencana_shift['jam_masuk'] >= df_rencana_shift['jam_pulang']

# df_rencana_shift.loc[mask, ['pulang_pre_time', 'pulang_post_time']] += pd.Timedelta(days=1)

df_rencana_shift.loc[df_rencana_shift['mask'], ['pulang_pre_time', 'pulang_post_time','jam_pulang']] = (
    df_rencana_shift.loc[df_rencana_shift['mask'], ['pulang_pre_time', 'pulang_post_time','jam_pulang']] + pd.Timedelta(days=1)
)

In [647]:
df_rencana_shift.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1744 entries, 0 to 1743
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   karyawan_id       1744 non-null   int64         
 1   instansi_id       1744 non-null   int64         
 2   tanggal_masuk     1744 non-null   datetime64[ns]
 3   shift_id          1744 non-null   int64         
 4   created_at        1744 non-null   datetime64[ns]
 5   editor_id         1744 non-null   int64         
 6   id                1744 non-null   int64         
 7   group_id          1744 non-null   int64         
 8   name              1744 non-null   object        
 9   masuk_pre_time    1744 non-null   datetime64[ns]
 10  masuk_post_time   1744 non-null   datetime64[ns]
 11  masuk_max_time    1744 non-null   datetime64[ns]
 12  pulang_pre_time   1744 non-null   datetime64[ns]
 13  pulang_post_time  1744 non-null   datetime64[ns]
 14  jam_masuk         1744 n

In [648]:
df_rencana_shift.head()

,karyawan_id,instansi_id,tanggal_masuk,shift_id,created_at,editor_id,id,group_id,name,masuk_pre_time,masuk_post_time,masuk_max_time,pulang_pre_time,pulang_post_time,jam_masuk,jam_pulang,mask
0,2470,3062,2025-10-01,1,2025-09-29 15:12:22,829,1,1,MDN01 - PEMKO MEDAN SENIN-KAMIS,2025-10-01 07:30:00,2025-10-01 08:15:59,2025-10-01 12:00:59,2025-10-01 16:30:00,2025-10-01 17:00:59,2025-10-01 08:00:00,2025-10-01 16:30:00,False
1,2470,3062,2025-10-02,1,2025-09-29 15:12:22,829,1,1,MDN01 - PEMKO MEDAN SENIN-KAMIS,2025-10-02 07:30:00,2025-10-02 08:15:59,2025-10-02 12:00:59,2025-10-02 16:30:00,2025-10-02 17:00:59,2025-10-02 08:00:00,2025-10-02 16:30:00,False
2,2470,3062,2025-10-03,2,2025-09-29 15:08:31,829,2,1,MDN02 - PEMKO MEDAN JUMAT,2025-10-03 07:30:00,2025-10-03 08:15:59,2025-10-03 12:00:59,2025-10-03 17:00:00,2025-10-03 17:30:59,2025-10-03 08:00:00,2025-10-03 17:00:00,False
3,2470,3062,2025-10-06,1,2025-09-29 15:12:22,829,1,1,MDN01 - PEMKO MEDAN SENIN-KAMIS,2025-10-06 07:30:00,2025-10-06 08:15:59,2025-10-06 12:00:59,2025-10-06 16:30:00,2025-10-06 17:00:59,2025-10-06 08:00:00,2025-10-06 16:30:00,False
4,2470,3062,2025-10-07,1,2025-09-29 15:12:22,829,1,1,MDN01 - PEMKO MEDAN SENIN-KAMIS,2025-10-07 07:30:00,2025-10-07 08:15:59,2025-10-07 12:00:59,2025-10-07 16:30:00,2025-10-07 17:00:59,2025-10-07 08:00:00,2025-10-07 16:30:00,False


In [649]:
df_rencana_shift[df_rencana_shift['karyawan_id'] == 32329]

,karyawan_id,instansi_id,tanggal_masuk,shift_id,created_at,editor_id,id,group_id,name,masuk_pre_time,masuk_post_time,masuk_max_time,pulang_pre_time,pulang_post_time,jam_masuk,jam_pulang,mask


#### Merge df_rencana + df_pegawai

In [650]:
df_R_S_P = df_rencana_shift.merge(df_pegawai, left_on='karyawan_id', right_on='id', suffixes=('_r', '_p'))

In [651]:
df_R_S_P.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1744 entries, 0 to 1743
Data columns (total 41 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   karyawan_id          1744 non-null   int64         
 1   instansi_id_r        1744 non-null   int64         
 2   tanggal_masuk        1744 non-null   datetime64[ns]
 3   shift_id             1744 non-null   int64         
 4   created_at_r         1744 non-null   datetime64[ns]
 5   editor_id            1744 non-null   int64         
 6   id_r                 1744 non-null   int64         
 7   group_id_r           1744 non-null   int64         
 8   name_r               1744 non-null   object        
 9   masuk_pre_time       1744 non-null   datetime64[ns]
 10  masuk_post_time      1744 non-null   datetime64[ns]
 11  masuk_max_time       1744 non-null   datetime64[ns]
 12  pulang_pre_time      1744 non-null   datetime64[ns]
 13  pulang_post_time     1744 non-nul

In [652]:
df_R_S_P.head()  # Menampilkan beberapa baris pertama dari DataFrame gabungan

,karyawan_id,instansi_id_r,tanggal_masuk,shift_id,created_at_r,editor_id,id_r,group_id_r,name_r,masuk_pre_time,...,golongan,kordinat,jabatan,eselon_id,pangkat_id,status_face,comment_face,verified_id,verified_date,presensi_face
0,2470,3062,2025-10-01,1,2025-09-29 15:12:22,829,1,1,MDN01 - PEMKO MEDAN SENIN-KAMIS,2025-10-01 07:30:00,...,Pembina Tk. I (IV/b),"{""lat"":""3.5578531"",""long"":""98.6464847"",""rad"":""...",Kepala Badan Kepegawaian dan Pengembangan Sumb...,4.0,14.0,0,None,NaN,NaT,1
1,2470,3062,2025-10-02,1,2025-09-29 15:12:22,829,1,1,MDN01 - PEMKO MEDAN SENIN-KAMIS,2025-10-02 07:30:00,...,Pembina Tk. I (IV/b),"{""lat"":""3.5578531"",""long"":""98.6464847"",""rad"":""...",Kepala Badan Kepegawaian dan Pengembangan Sumb...,4.0,14.0,0,None,NaN,NaT,1
2,2470,3062,2025-10-03,2,2025-09-29 15:08:31,829,2,1,MDN02 - PEMKO MEDAN JUMAT,2025-10-03 07:30:00,...,Pembina Tk. I (IV/b),"{""lat"":""3.5578531"",""long"":""98.6464847"",""rad"":""...",Kepala Badan Kepegawaian dan Pengembangan Sumb...,4.0,14.0,0,None,NaN,NaT,1
3,2470,3062,2025-10-06,1,2025-09-29 15:12:22,829,1,1,MDN01 - PEMKO MEDAN SENIN-KAMIS,2025-10-06 07:30:00,...,Pembina Tk. I (IV/b),"{""lat"":""3.5578531"",""long"":""98.6464847"",""rad"":""...",Kepala Badan Kepegawaian dan Pengembangan Sumb...,4.0,14.0,0,None,NaN,NaT,1
4,2470,3062,2025-10-07,1,2025-09-29 15:12:22,829,1,1,MDN01 - PEMKO MEDAN SENIN-KAMIS,2025-10-07 07:30:00,...,Pembina Tk. I (IV/b),"{""lat"":""3.5578531"",""long"":""98.6464847"",""rad"":""...",Kepala Badan Kepegawaian dan Pengembangan Sumb...,4.0,14.0,0,None,NaN,NaT,1


#### Convert to DateTime

In [653]:
first_date = pd.Timestamp(tanggal_awal)
last_date = pd.Timestamp(tanggal_akhir)

first_date, last_date

(Timestamp('2025-10-01 00:00:00'), Timestamp('2025-10-31 00:00:00'))

In [654]:
# keep datetime64[ns], do NOT convert to python date objects
df_presensi['tanggal_masuk'] = pd.to_datetime(df_presensi['tanggal_masuk'])

In [655]:
print("dtype:", df_presensi['tanggal_masuk'].dtype)
print(df_presensi['tanggal_masuk'].head(10))
# show element types for the first few values
print([type(x) for x in df_presensi['tanggal_masuk'].head(10)])

dtype: datetime64[ns]
0   2025-10-28
1   2025-10-08
2   2025-10-10
3   2025-10-10
4   2025-10-20
5   2025-10-03
6   2025-10-20
7   2025-10-01
8   2025-10-23
9   2025-10-08
Name: tanggal_masuk, dtype: datetime64[ns]
[<class 'pandas._libs.tslibs.timestamps.Timestamp'>, <class 'pandas._libs.tslibs.timestamps.Timestamp'>, <class 'pandas._libs.tslibs.timestamps.Timestamp'>, <class 'pandas._libs.tslibs.timestamps.Timestamp'>, <class 'pandas._libs.tslibs.timestamps.Timestamp'>, <class 'pandas._libs.tslibs.timestamps.Timestamp'>, <class 'pandas._libs.tslibs.timestamps.Timestamp'>, <class 'pandas._libs.tslibs.timestamps.Timestamp'>, <class 'pandas._libs.tslibs.timestamps.Timestamp'>, <class 'pandas._libs.tslibs.timestamps.Timestamp'>]


In [656]:
# run in a cell before calling generate_presensi_laporan
import datetime
print("type of tanggal_kerja (example from df_rencana):", type(df_rencana['tanggal_masuk'].iloc[0]))
print("dtype df_rencana['tanggal_masuk']:", df_rencana['tanggal_masuk'].dtype)
print("dtype df_presensi['tanggal_masuk']:", df_presensi['tanggal_masuk'].dtype)

type of tanggal_kerja (example from df_rencana): <class 'datetime.date'>
dtype df_rencana['tanggal_masuk']: object
dtype df_presensi['tanggal_masuk']: datetime64[ns]


## Generate Data Laporan

### def generate_presensi_laporan

In [657]:
def generate_presensi_laporan(df_pegawai, df_rencana, df_presensi, df_absen, bulan_cetak, tahun_cetak, tanggal_awal, tanggal_akhir):
    # Gabungkan karyawan + rencana (inner join)
    df = df_rencana.merge(df_pegawai, left_on='karyawan_id', right_on='id', suffixes=('_r', '_k'))

    laporan_rows = []

    # Loop per rencana kerja
    for _, row in df.iterrows():
        karyawan_id = row['karyawan_id']
        instansi_id = row['instansi_id_r']
        tanggal_kerja = row['tanggal_masuk']
        jadwal_masuk = row['masuk_post_time']
        jadwal_pulang = row['pulang_pre_time']

        # inside loop in generate_presensi_laporan
        # ensure tanggal_kerja is a Timestamp
        tanggal_kerja_ts = pd.Timestamp(tanggal_kerja)  # handles Timestamp, str, datetime.date, etc.
        start_of_day = tanggal_kerja_ts.normalize()           # e.g. 2025-09-01 00:00:00
        end_of_day = start_of_day + pd.Timedelta(days=1)      # exclusive end

        # Filter absen (izin/cuti) untuk karyawan tsb
        df_absen_karyawan = df_absen[df_absen['karyawan_id'] == karyawan_id]
        absen_row = df_absen_karyawan[
            (df_absen_karyawan['tanggal_mulai'] <= tanggal_kerja) &
            (df_absen_karyawan['tanggal_selesai'] >= tanggal_kerja)
        ]

        if not absen_row.empty:
            # Karyawan sedang absen
            keterangan_absen = absen_row.iloc[0]['type']
            jam_masuk = None
            jam_pulang = None
            keterangan_hadir = None
        else:

            # Cek presendf_presensi
            df_presensi_karyawan = df_presensi[df_presensi['karyawan_id'] == karyawan_id]

            # Masuk: compare using a half-open range
            masuk = df_presensi_karyawan[
                (df_presensi_karyawan['jenis'] == 'M') &
                (df_presensi_karyawan['tanggal_masuk'] >= start_of_day) &
                (df_presensi_karyawan['tanggal_masuk'] < end_of_day) &
                (df_presensi_karyawan['approver_status'].isin([None, 'TERIMA']))
            ].sort_values('tanggal_kirim', ascending=True).head(1)

            jam_masuk = masuk['tanggal_kirim'].iloc[0] if not masuk.empty else None

            # Pulang
            pulang = df_presensi_karyawan[
                (df_presensi_karyawan['jenis'] == 'P') &
                (df_presensi_karyawan['tanggal_masuk'] >= start_of_day) &
                (df_presensi_karyawan['tanggal_masuk'] < end_of_day) &
                (df_presensi_karyawan['approver_status'].isin([None, 'TERIMA']))
            ].sort_values('tanggal_kirim', ascending=False).head(1)

            jam_pulang = pulang['tanggal_kirim'].iloc[0] if not pulang.empty else None

            keterangan_hadir = (
                masuk['catatan'].iloc[0]
                if not masuk.empty and pd.notna(masuk['catatan'].iloc[0])
                else (pulang['catatan'].iloc[0] if not pulang.empty else None)
            )
            keterangan_absen = None

        laporan_rows.append({
            'karyawan_id': karyawan_id,
            'instansi_id': instansi_id,
            'tanggal_kerja': tanggal_kerja,
            'jadwal_masuk': jadwal_masuk,
            'jadwal_pulang': jadwal_pulang,
            'jam_masuk': jam_masuk,
            'jam_pulang': jam_pulang,
            'keterangan_hadir': keterangan_hadir,
            'keterangan_absen': keterangan_absen
        })

    # Konversi hasil ke DataFrame
    df_laporan = pd.DataFrame(laporan_rows)
    return df_laporan

### Running Function

In [658]:
df_laporan = generate_presensi_laporan(df_pegawai, df_rencana_shift, df_presensi, df_absen, 9, 2025, first_date, last_date)

### Hasil Olah Data

In [659]:
df_laporan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1744 entries, 0 to 1743
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   karyawan_id       1744 non-null   int64         
 1   instansi_id       1744 non-null   int64         
 2   tanggal_kerja     1744 non-null   datetime64[ns]
 3   jadwal_masuk      1744 non-null   datetime64[ns]
 4   jadwal_pulang     1744 non-null   datetime64[ns]
 5   jam_masuk         1493 non-null   datetime64[ns]
 6   jam_pulang        1514 non-null   datetime64[ns]
 7   keterangan_hadir  1572 non-null   object        
 8   keterangan_absen  118 non-null    object        
dtypes: datetime64[ns](5), int64(2), object(2)
memory usage: 122.8+ KB


#### Cek duplikat

In [660]:
# Periksa apakah ada yang duplicate untuk kombinasi karyawan_id + tanggal_kerja
duplicates = df_laporan[df_laporan.duplicated(subset=['karyawan_id', 'tanggal_kerja'], keep=False)]
if not duplicates.empty:
    print("Terdapat duplikat untuk pegawai dan tanggal kerja yang sama:")
    print(duplicates[['karyawan_id', 'tanggal_kerja']])
else:
    print("Tidak ada data duplikat untuk pegawai dan tanggal kerja yang sama.")

Tidak ada data duplikat untuk pegawai dan tanggal kerja yang sama.


#### Pegawai tertentu

In [661]:
df_laporan[df_laporan['karyawan_id'] == 14108]

,karyawan_id,instansi_id,tanggal_kerja,jadwal_masuk,jadwal_pulang,jam_masuk,jam_pulang,keterangan_hadir,keterangan_absen
46,14108,3062,2025-10-01,2025-10-01 08:29:59,2025-10-01 17:00:00,2025-10-01 08:04:29,2025-10-01 17:02:33,,None
47,14108,3062,2025-10-02,2025-10-02 08:29:59,2025-10-02 17:00:00,2025-10-02 08:13:17,2025-10-02 17:06:22,,None
48,14108,3062,2025-10-03,2025-10-03 08:29:59,2025-10-03 17:30:00,2025-10-03 08:15:20,2025-10-03 17:31:05,,None
49,14108,3062,2025-10-06,2025-10-06 08:15:59,2025-10-06 16:30:00,NaT,2025-10-06 16:31:55,SPT STULA PKP Samosir,None
50,14108,3062,2025-10-07,2025-10-07 08:29:59,2025-10-07 17:00:00,2025-10-07 08:02:27,2025-10-07 17:01:43,SPT PKP Samosir,None
51,14108,3062,2025-10-08,2025-10-08 08:29:59,2025-10-08 17:00:00,2025-10-08 08:02:11,2025-10-08 17:01:57,SPT PKP SOMOSIR,None
52,14108,3062,2025-10-09,2025-10-09 08:29:59,2025-10-09 17:00:00,2025-10-09 08:20:39,2025-10-09 17:02:54,,None
53,14108,3062,2025-10-10,2025-10-10 08:29:59,2025-10-10 17:30:00,2025-10-10 08:06:00,2025-10-10 17:34:22,SPT PKP P4TK,None
54,14108,3062,2025-10-13,2025-10-13 08:15:59,2025-10-13 16:30:00,2025-10-13 07:50:16,2025-10-13 16:43:31,,None
55,14108,3062,2025-10-14,2025-10-14 08:29:59,2025-10-14 17:00:00,NaT,NaT,None,S


#### Keterangan Hadir Not Null

In [662]:
# Tampilkan data laporan dengan keterangan hadir tidak kosong atau teks None
df_laporan_non_empty = df_laporan[
    (df_laporan['keterangan_hadir'].notna()) &
    (df_laporan['keterangan_hadir'].str.strip().astype(bool))
]

df_laporan_non_empty

,karyawan_id,instansi_id,tanggal_kerja,jadwal_masuk,jadwal_pulang,jam_masuk,jam_pulang,keterangan_hadir,keterangan_absen
49,14108,3062,2025-10-06,2025-10-06 08:15:59,2025-10-06 16:30:00,NaT,2025-10-06 16:31:55,SPT STULA PKP Samosir,None
50,14108,3062,2025-10-07,2025-10-07 08:29:59,2025-10-07 17:00:00,2025-10-07 08:02:27,2025-10-07 17:01:43,SPT PKP Samosir,None
51,14108,3062,2025-10-08,2025-10-08 08:29:59,2025-10-08 17:00:00,2025-10-08 08:02:11,2025-10-08 17:01:57,SPT PKP SOMOSIR,None
53,14108,3062,2025-10-10,2025-10-10 08:29:59,2025-10-10 17:30:00,2025-10-10 08:06:00,2025-10-10 17:34:22,SPT PKP P4TK,None
85,14472,3062,2025-10-23,2025-10-23 08:29:59,2025-10-23 17:00:00,2025-10-23 08:02:03,2025-10-23 17:09:33,mengikuti uji coba asesmen BKN kanreg VI medan,None
...,...,...,...,...,...,...,...,...,...
1738,32760,3062,2025-10-24,2025-10-24 08:29:59,2025-10-24 17:30:00,2025-10-24 08:02:20,2025-10-24 17:52:37,spt adc walikota medan,None
1740,32760,3062,2025-10-28,2025-10-28 08:29:59,2025-10-28 17:00:00,2025-10-28 08:36:51,2025-10-28 17:10:35,spt adc walikota medan,None
1741,32760,3062,2025-10-29,2025-10-29 08:29:59,2025-10-29 17:00:00,2025-10-29 08:35:05,2025-10-29 17:13:22,spt adc walikota medan,None
1742,32760,3062,2025-10-30,2025-10-30 08:29:59,2025-10-30 17:00:00,2025-10-30 08:00:30,2025-10-30 17:07:24,spt adc walikota medan,None


#### Keterangan Absen Not Null

In [663]:
# Tampilkan data laporan dengan keterangan absen tidak null atau teks None
df_laporan_absen_non_empty = df_laporan[
    (df_laporan['keterangan_absen'].notna()) &
    (df_laporan['keterangan_absen'].str.strip().astype(bool))
]
df_laporan_absen_non_empty

,karyawan_id,instansi_id,tanggal_kerja,jadwal_masuk,jadwal_pulang,jam_masuk,jam_pulang,keterangan_hadir,keterangan_absen
55,14108,3062,2025-10-14,2025-10-14 08:29:59,2025-10-14 17:00:00,NaT,NaT,None,S
61,14108,3062,2025-10-22,2025-10-22 08:29:59,2025-10-22 17:00:00,NaT,NaT,None,C
62,14108,3062,2025-10-23,2025-10-23 08:29:59,2025-10-23 17:00:00,NaT,NaT,None,C
63,14108,3062,2025-10-24,2025-10-24 08:29:59,2025-10-24 17:30:00,NaT,NaT,None,C
64,14108,3062,2025-10-27,2025-10-27 08:15:59,2025-10-27 16:30:00,NaT,NaT,None,C
...,...,...,...,...,...,...,...,...,...
1620,31685,3062,2025-10-15,2025-10-15 08:29:59,2025-10-15 17:00:00,NaT,NaT,None,C
1705,32624,3062,2025-10-06,2025-10-06 08:15:59,2025-10-06 16:30:00,NaT,NaT,None,S
1715,32624,3062,2025-10-20,2025-10-20 08:15:59,2025-10-20 16:30:00,NaT,NaT,None,C
1716,32624,3062,2025-10-21,2025-10-21 08:29:59,2025-10-21 17:00:00,NaT,NaT,None,C


## Rekapitulasi Kehadiran

### Rekapitulasi Bulanan

In [664]:
from datetime import datetime, timedelta

#### Olah Data

In [665]:
# Menggabungkan data laporan dengan data pegawai untuk mendapatkan nama pegawai
df_laporan_bulanan = df_laporan.merge(df_pegawai[['id', 'nip', 'name']], left_on='karyawan_id', right_on='id', how='left', suffixes=('', '_pegawai'))

# Cetak df_laporan per pegawai ditampilkan hanya satu baris per kombinasi nip, name, tanggal_kerja, keterangan_hadir, keterangan_absen
df_laporan_bulanan[['nip', 'name']].drop_duplicates().head()

,nip,name
0,198412222003121001,"SUBHAN FAJRI HARAHAP, S.STP, M.A.P"
23,196803191996022001,Ir NETTI EFRIDAWATI PURBA
46,196804041993032006,"NUR ASIAH RANGKUTI, SH"
69,198611062011012007,"RIDHA NOVIANA HARAHAP, S.Kom, M.I.Kom"
92,197304212009022001,"ASLAMIAH, S.E."


In [666]:
df_laporan_bulanan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1744 entries, 0 to 1743
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   karyawan_id       1744 non-null   int64         
 1   instansi_id       1744 non-null   int64         
 2   tanggal_kerja     1744 non-null   datetime64[ns]
 3   jadwal_masuk      1744 non-null   datetime64[ns]
 4   jadwal_pulang     1744 non-null   datetime64[ns]
 5   jam_masuk         1493 non-null   datetime64[ns]
 6   jam_pulang        1514 non-null   datetime64[ns]
 7   keterangan_hadir  1572 non-null   object        
 8   keterangan_absen  118 non-null    object        
 9   id                1744 non-null   int64         
 10  nip               1744 non-null   object        
 11  name              1744 non-null   object        
dtypes: datetime64[ns](5), int64(3), object(4)
memory usage: 163.6+ KB


In [667]:
df_laporan_bulanan.head()

,karyawan_id,instansi_id,tanggal_kerja,jadwal_masuk,jadwal_pulang,jam_masuk,jam_pulang,keterangan_hadir,keterangan_absen,id,nip,name
0,2470,3062,2025-10-01,2025-10-01 08:15:59,2025-10-01 16:30:00,NaT,NaT,None,None,2470,198412222003121001,"SUBHAN FAJRI HARAHAP, S.STP, M.A.P"
1,2470,3062,2025-10-02,2025-10-02 08:15:59,2025-10-02 16:30:00,NaT,NaT,None,None,2470,198412222003121001,"SUBHAN FAJRI HARAHAP, S.STP, M.A.P"
2,2470,3062,2025-10-03,2025-10-03 08:15:59,2025-10-03 17:00:00,NaT,NaT,None,None,2470,198412222003121001,"SUBHAN FAJRI HARAHAP, S.STP, M.A.P"
3,2470,3062,2025-10-06,2025-10-06 08:15:59,2025-10-06 16:30:00,NaT,NaT,None,None,2470,198412222003121001,"SUBHAN FAJRI HARAHAP, S.STP, M.A.P"
4,2470,3062,2025-10-07,2025-10-07 08:15:59,2025-10-07 16:30:00,NaT,NaT,None,None,2470,198412222003121001,"SUBHAN FAJRI HARAHAP, S.STP, M.A.P"


In [668]:
# convert jam_masuk, jam_pulang, jadwal_masuk, jadwal_pulang ke datatime64[ns]
df_laporan_bulanan['jam_masuk'] = pd.to_datetime(df_laporan_bulanan['jam_masuk'])
df_laporan_bulanan['jam_pulang'] = pd.to_datetime(df_laporan_bulanan['jam_pulang'])
df_laporan_bulanan['jadwal_masuk'] = pd.to_datetime(df_laporan_bulanan['jadwal_masuk'])
df_laporan_bulanan['jadwal_pulang'] = pd.to_datetime(df_laporan_bulanan['jadwal_pulang'])

#### Def Fungsi

In [669]:
# def carbon_parse(time_str, add_minutes):
#     """Geser waktu (HH:MM:SS) sebanyak add_minutes"""
#     if pd.isna(time_str):
#         return None
#     # t = datetime.strptime(time_str, '%H:%M:%S')
#     t = pd.to_datetime(time_str)
#     t += timedelta(minutes=add_minutes)
#     # return t.strftime('%H:%M:%S')
#     return t

def carbon_parse(time_str, add_minutes):
    if pd.isna(time_str):
        return None
    t = pd.to_datetime(time_str)
    t += timedelta(minutes=add_minutes)
    return t


def masuk_kategori(row):
    """Menentukan kategori keterlambatan masuk"""
    jm, jadwal = row['jam_masuk'], row['jadwal_masuk']
    if pd.isna(jadwal):
        return None
    if pd.isna(jm):
        return 't4' if not pd.isna(row['jam_pulang']) else None
    if jm <= jadwal:
        return 'tw'
    elif jadwal < jm <= carbon_parse(jadwal, 30):
        return 't1'
    elif carbon_parse(jadwal, 30) < jm <= carbon_parse(jadwal, 60):
        return 't2'
    elif carbon_parse(jadwal, 60) < jm <= carbon_parse(jadwal, 90):
        return 't3'
    else:
        return 't4'

def pulang_kategori(row):
    """Menentukan kategori kepulangan"""
    jp, jadwal = row['jam_pulang'], row['jadwal_pulang']
    if pd.isna(jadwal):
        return None
    if pd.isna(jp):
        return 'p4' if not pd.isna(row['jam_masuk']) else None
    if jp >= jadwal:
        return 'tw'
    elif carbon_parse(jadwal, -30) <= jp < jadwal:
        return 'p1'
    elif carbon_parse(jadwal, -60) <= jp < carbon_parse(jadwal, -30):
        return 'p2'
    elif carbon_parse(jadwal, -90) <= jp < carbon_parse(jadwal, -60):
        return 'p3'
    else:
        return 'p4'

def status_hadir(row):
    """Menandai kehadiran"""
    if not pd.isna(row['jam_masuk']) or not pd.isna(row['jam_pulang']):
        return 'hadir'
    elif row['keterangan_absen'] in ['C', 'S']:
        return 'izin/sakit'
    elif row['keterangan_absen'] in ['TB', 'BK']:
        return 'tugas/bk'
    else:
        return 'tidak hadir'


In [670]:
# --- normalisasi kolom waktu sebelum apply fungsi ---
# for col in ['jam_masuk', 'jadwal_masuk', 'jam_pulang', 'jadwal_pulang']:
#     if pd.api.types.is_timedelta64_dtype(df_laporan_bulanan[col]):
#         # ubah timedelta jadi format 'HH:MM:SS'
#         df_laporan_bulanan[col] = df_laporan_bulanan[col].apply(lambda x: str(x).split(' ')[-1] if pd.notna(x) else None)
#     else:
#         # ubah string yang masih ada tulisan "0 days " di depannya
#         df_laporan_bulanan[col] = df_laporan_bulanan[col].astype(str).str.replace(r'^0 days ', '', regex=True)
#         df_laporan_bulanan.loc[df_laporan_bulanan[col].isin(['None', 'NaT', 'nan']), col] = None


In [671]:
# pastikan kolom waktu dalam format HH:MM:SS string
# for col in ['jam_masuk', 'jadwal_masuk', 'jam_pulang', 'jadwal_pulang']:
#     df_laporan_bulanan[col] = df_laporan_bulanan[col].astype(str)
#     df_laporan_bulanan.loc[df_laporan_bulanan[col] == 'None', col] = None

# tambahkan kolom kategori
df_laporan_bulanan['masuk_kat'] = df_laporan_bulanan.apply(masuk_kategori, axis=1)
df_laporan_bulanan['pulang_kat'] = df_laporan_bulanan.apply(pulang_kategori, axis=1)
df_laporan_bulanan['status_hadir'] = df_laporan_bulanan.apply(status_hadir, axis=1)

#### Hitung Rekap

In [672]:
df_laporan_bulanan[df_laporan_bulanan['karyawan_id'] == 22777].head(5)

,karyawan_id,instansi_id,tanggal_kerja,jadwal_masuk,jadwal_pulang,jam_masuk,jam_pulang,keterangan_hadir,keterangan_absen,id,nip,name,masuk_kat,pulang_kat,status_hadir
736,22777,3062,2025-10-01,2025-10-01 08:29:59,2025-10-01 17:00:00,2025-10-01 08:02:10,2025-10-01 17:01:05,,None,22777,198905142011011004,"HASNAN AULIA HAQ, S.Si",tw,tw,hadir
737,22777,3062,2025-10-02,2025-10-02 08:29:59,2025-10-02 17:00:00,2025-10-02 08:29:19,2025-10-02 17:18:54,,None,22777,198905142011011004,"HASNAN AULIA HAQ, S.Si",tw,tw,hadir
738,22777,3062,2025-10-03,2025-10-03 08:29:59,2025-10-03 17:30:00,2025-10-03 08:24:34,2025-10-03 17:35:20,,None,22777,198905142011011004,"HASNAN AULIA HAQ, S.Si",tw,tw,hadir
739,22777,3062,2025-10-06,2025-10-06 08:15:59,2025-10-06 16:30:00,2025-10-06 08:00:15,2025-10-06 16:34:16,,None,22777,198905142011011004,"HASNAN AULIA HAQ, S.Si",tw,tw,hadir
740,22777,3062,2025-10-07,2025-10-07 08:29:59,2025-10-07 17:00:00,2025-10-07 08:14:55,2025-10-07 17:07:38,,None,22777,198905142011011004,"HASNAN AULIA HAQ, S.Si",tw,tw,hadir


In [673]:
rekap_per_pegawai = df_laporan_bulanan.groupby(['karyawan_id','nip', 'name']).agg(
    jumlah_hari=('tanggal_kerja', 'count'),
    hadir=('status_hadir', lambda x: (x == 'hadir').sum()),
    tidak_hadir=('status_hadir', lambda x: (x != 'hadir').sum()),
    twm=('masuk_kat', lambda x: (x == 'tw').sum()),
    t1=('masuk_kat', lambda x: (x == 't1').sum()),
    t2=('masuk_kat', lambda x: (x == 't2').sum()),
    t3=('masuk_kat', lambda x: (x == 't3').sum()),
    t4=('masuk_kat', lambda x: (x == 't4').sum()),
    twp=('pulang_kat', lambda x: (x == 'tw').sum()),
    p1=('pulang_kat', lambda x: (x == 'p1').sum()),
    p2=('pulang_kat', lambda x: (x == 'p2').sum()),
    p3=('pulang_kat', lambda x: (x == 'p3').sum()),
    p4=('pulang_kat', lambda x: (x == 'p4').sum()),
    izin_sakit=('status_hadir', lambda x: (x == 'izin/sakit').sum()),
    tugas_bk=('status_hadir', lambda x: (x == 'tugas/bk').sum()),
    tanpa_keterangan=('status_hadir', lambda x: (x == 'tidak hadir').sum()),
).reset_index()

display(rekap_per_pegawai)


,karyawan_id,nip,name,jumlah_hari,hadir,tidak_hadir,twm,t1,t2,t3,t4,twp,p1,p2,p3,p4,izin_sakit,tugas_bk,tanpa_keterangan
0,2470,198412222003121001,"SUBHAN FAJRI HARAHAP, S.STP, M.A.P",23,0,23,0,0,0,0,0,0,0,0,0,0,0,0,23
1,14080,196803191996022001,Ir NETTI EFRIDAWATI PURBA,23,23,0,21,0,0,0,2,23,0,0,0,0,0,0,0
2,14108,196804041993032006,"NUR ASIAH RANGKUTI, SH",23,15,8,14,0,0,0,1,12,0,0,0,3,7,0,1
3,14472,198611062011012007,"RIDHA NOVIANA HARAHAP, S.Kom, M.I.Kom",23,23,0,21,0,0,0,2,20,0,0,0,3,0,0,0
4,15486,197304212009022001,"ASLAMIAH, S.E.",23,23,0,23,0,0,0,0,23,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,32220,199104162025211006,AIDAL SAFITRA,23,23,0,20,2,1,0,0,23,0,0,0,0,0,0,0
72,32395,198610272025212003,PUTRI SYARAH LUBIS,23,23,0,23,0,0,0,0,23,0,0,0,0,0,0,0
73,32553,199301242025211004,"AHMAD NOVAL, S.Kom",23,23,0,23,0,0,0,0,23,0,0,0,0,0,0,0
74,32624,199004012025212005,"KIKI RIZKY ALVONSA, S.Kom",23,19,4,19,0,0,0,0,19,0,0,0,0,4,0,0


In [674]:
from IPython.display import display, HTML

display(HTML(rekap_per_pegawai.to_html(max_rows=None, max_cols=None)))

,karyawan_id,nip,name,jumlah_hari,hadir,tidak_hadir,twm,t1,t2,t3,t4,twp,p1,p2,p3,p4,izin_sakit,tugas_bk,tanpa_keterangan
0,2470,198412222003121001,"SUBHAN FAJRI HARAHAP, S.STP, M.A.P",23,0,23,0,0,0,0,0,0,0,0,0,0,0,0,23
1,14080,196803191996022001,Ir NETTI EFRIDAWATI PURBA,23,23,0,21,0,0,0,2,23,0,0,0,0,0,0,0
2,14108,196804041993032006,"NUR ASIAH RANGKUTI, SH",23,15,8,14,0,0,0,1,12,0,0,0,3,7,0,1
3,14472,198611062011012007,"RIDHA NOVIANA HARAHAP, S.Kom, M.I.Kom",23,23,0,21,0,0,0,2,20,0,0,0,3,0,0,0
4,15486,197304212009022001,"ASLAMIAH, S.E.",23,23,0,23,0,0,0,0,23,0,0,0,0,0,0,0
5,16443,197510181998032003,HIKMAH SIMBOLON,23,17,6,11,3,1,0,2,16,0,0,0,1,5,0,1
6,16640,197608092005022003,"LINDA AGUSTINA SIREGAR, SS, M.A.P",23,22,1,19,0,0,0,3,22,0,0,0,0,1,0,0
7,16790,197704082006042004,"EVI RAHIMI AMRI, S.Psi.",23,17,6,14,3,0,0,0,17,0,0,0,0,6,0,0
8,17172,197809152010012008,"KARTIKA MAHDALENA, S.E.",23,21,2,15,3,3,0,0,20,0,0,0,1,2,0,0
9,17660,196906051997032001,"NURMA SATYARINI, S.H.",23,21,2,18,2,0,0,1,21,0,0,0,0,2,0,0


#### Simpan hasil ke format Excel

In [675]:
# Simpan rekapitulasi ke file Excel
output_filename = f"rekapitulasi_kehadiran_{instansi_id}_{bulan_cetak}_{tahun_cetak}.xlsx"
rekap_per_pegawai.to_excel(output_filename, index=False)
print(f"Rekapitulasi kehadiran disimpan ke file: {output_filename}")

Rekapitulasi kehadiran disimpan ke file: rekapitulasi_kehadiran_3062_10_2025.xlsx


#### Debug Hasil

In [680]:
karyawan_id_test = 22777

In [681]:
rekap_per_pegawai[rekap_per_pegawai['karyawan_id'] == karyawan_id_test]

,karyawan_id,nip,name,jumlah_hari,hadir,tidak_hadir,twm,t1,t2,t3,t4,twp,p1,p2,p3,p4,izin_sakit,tugas_bk,tanpa_keterangan
32,22777,198905142011011004,"HASNAN AULIA HAQ, S.Si",23,23,0,21,1,0,0,1,22,0,0,0,1,0,0,0


In [682]:
# menampilkan detail kehadiran bulanan untuk karyawan tertentu
# hilangkan kolom instansi_id, id, nip, name 
df_laporan_bulanan[df_laporan_bulanan['karyawan_id'] == karyawan_id_test].drop(columns=['instansi_id', 'id', 'nip', 'name'])

,karyawan_id,tanggal_kerja,jadwal_masuk,jadwal_pulang,jam_masuk,jam_pulang,keterangan_hadir,keterangan_absen,masuk_kat,pulang_kat,status_hadir
736,22777,2025-10-01,2025-10-01 08:29:59,2025-10-01 17:00:00,2025-10-01 08:02:10,2025-10-01 17:01:05,,None,tw,tw,hadir
737,22777,2025-10-02,2025-10-02 08:29:59,2025-10-02 17:00:00,2025-10-02 08:29:19,2025-10-02 17:18:54,,None,tw,tw,hadir
738,22777,2025-10-03,2025-10-03 08:29:59,2025-10-03 17:30:00,2025-10-03 08:24:34,2025-10-03 17:35:20,,None,tw,tw,hadir
739,22777,2025-10-06,2025-10-06 08:15:59,2025-10-06 16:30:00,2025-10-06 08:00:15,2025-10-06 16:34:16,,None,tw,tw,hadir
740,22777,2025-10-07,2025-10-07 08:29:59,2025-10-07 17:00:00,2025-10-07 08:14:55,2025-10-07 17:07:38,,None,tw,tw,hadir
741,22777,2025-10-08,2025-10-08 08:29:59,2025-10-08 17:00:00,2025-10-08 08:28:16,2025-10-08 17:06:33,,None,tw,tw,hadir
742,22777,2025-10-09,2025-10-09 08:29:59,2025-10-09 17:00:00,2025-10-09 08:13:55,2025-10-09 17:01:36,,None,tw,tw,hadir
743,22777,2025-10-10,2025-10-10 08:29:59,2025-10-10 17:30:00,2025-10-10 08:23:34,2025-10-10 17:42:12,,None,tw,tw,hadir
744,22777,2025-10-13,2025-10-13 08:15:59,2025-10-13 16:30:00,2025-10-13 08:14:35,2025-10-13 16:41:05,,None,tw,tw,hadir
745,22777,2025-10-14,2025-10-14 08:29:59,2025-10-14 17:00:00,2025-10-14 08:00:41,2025-10-14 17:16:07,,None,tw,tw,hadir


In [683]:
# Ekspor detail kehadiran bulanan karyawan tertentu ke file Excel
output_detail_filename = f"detail_kehadiran_karyawan_{karyawan_id_test}_{bulan_cetak}_{tahun_cetak}.xlsx"
df_laporan_bulanan[df_laporan_bulanan['karyawan_id'] == karyawan_id_test].drop(columns=['instansi_id', 'id', 'nip', 'name']).to_excel(output_detail_filename, index=False)
print(f"Detail kehadiran bulanan karyawan disimpan ke file: {output_detail_filename}")


Detail kehadiran bulanan karyawan disimpan ke file: detail_kehadiran_karyawan_22777_10_2025.xlsx
